In [1]:
import re
import tensorflow
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, SpatialDropout1D
from keras.models import Sequential
from keras.callbacks import EarlyStopping
import pandas as pd
import numpy as np
import simplemma
from stop_words import get_stop_words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use("ggplot")

In [2]:
stop_words = get_stop_words('uk')

In [3]:
#попередня обробка тексту
def ClearText(text):
    #переведення до нижнього регістру всіх слів
    cleartext = text.lower()
    #print(cleartext)
    #прибирання пустих рядків та розрив рядків
    cleartext = re.sub('\-\s\r\n\s{1,}|\-\s\r\n|\r\n', '', cleartext) 
    #залишаємо лише слова, прибираємо пунктуацію та числа
    cleartext = re.sub('[.,:;_%©?*,!@#$%^&()\d]|[+=]|[[]|[]]|[/]|"|\s{2,}|-', ' ', cleartext) #deleting symbols  
    #cleartext = cleartext.translate(remove_digits)
    cleartext = cleartext.replace("\\", "")
    cleartext = cleartext.rstrip()
    #прибираємо зайві пробіи
    cleartext = re.sub(" +", " ", cleartext)
    #ділимо речення на список слів, розбиваємо по пробілам
    cleartext = re.split(" ", cleartext)
    #прибираємо стопслова
    cleartext = [word for word in cleartext if word not in stop_words]
    #прибираємо слова, довжина який менше 3 букв
    cleartext = [word for word in cleartext if len(word) > 3]
    #лематизація слів
    cleartext = [simplemma.lemmatize(word, lang='uk') for word in cleartext]
    cleartext = [word.lower() for word in cleartext ]
    return ' '.join(cleartext)

In [11]:
df = pd.read_excel('D:\\NLP\\NLP\\NLP_all.xlsx')
df = df.sample(frac=1) 

MAX_NB_WORDS = 50000
MAX_SEQUENCE_LENGTH = 100
EMBEDDING_DIM = 50

df['Category'] = df['Category'].replace(['Toxic', 'Not_Toxic'],[1,0])
df.columns = ['Comment', 'Category']
df_train, df_test = train_test_split(df, test_size = 0.10)


# Transformers

In [33]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from simpletransformers.classification import ClassificationModel, ClassificationArgs

model_args = ClassificationArgs(num_train_epochs=1, overwrite_output_dir=True, manual_seed=42)
model = ClassificationModel(model_type='roberta', model_name='roberta-base', use_cuda=False, num_labels=2, args=model_args)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

In [34]:
model.train_model(df_train)

  0%|          | 0/1046 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/131 [00:00<?, ?it/s]

(131, 0.6772095902275493)

In [54]:
result, model_outputs, wrong_preds = model.eval_model(df_test)
print(result)
print(model_outputs)
print(wrong_preds)

c:\Users\Максим\AppData\Local\Programs\Python\Python310\lib\site-packages\simpletransformers\classification\classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/117 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/15 [00:00<?, ?it/s]

{'mcc': 0.0, 'tp': 0, 'tn': 78, 'fp': 0, 'fn': 39, 'auroc': 0.5631163708086785, 'auprc': 0.382683485614188, 'eval_loss': 0.6496504783630371}
[[ 0.21981834 -0.14733961]
 [ 0.21858491 -0.12934802]
 [ 0.21960326 -0.13558047]
 [ 0.22822918 -0.14114752]
 [ 0.22353126 -0.13419114]
 [ 0.21151285 -0.13792281]
 [ 0.2175865  -0.13173421]
 [ 0.22569893 -0.1351627 ]
 [ 0.22624062 -0.13916467]
 [ 0.21662267 -0.13627145]
 [ 0.21533553 -0.1436439 ]
 [ 0.21918668 -0.1372977 ]
 [ 0.21540926 -0.13897908]
 [ 0.21839978 -0.13385932]
 [ 0.22628587 -0.13728803]
 [ 0.22001745 -0.13438225]
 [ 0.21569352 -0.13399921]
 [ 0.22522973 -0.13993819]
 [ 0.21862902 -0.13470228]
 [ 0.21481565 -0.13194957]
 [ 0.22196242 -0.1382786 ]
 [ 0.22120221 -0.13723899]
 [ 0.22029175 -0.13243844]
 [ 0.22142689 -0.14056478]
 [ 0.21674305 -0.13519867]
 [ 0.22232632 -0.13436984]
 [ 0.2254737  -0.1392775 ]
 [ 0.21780412 -0.13310818]
 [ 0.22049092 -0.13581191]
 [ 0.22408588 -0.14202717]
 [ 0.21627523 -0.13297813]
 [ 0.21652229 -0.14099

In [52]:
predictions = []
for x in model_outputs:
    predictions.append(np.argmax(x))
print('f1 score:', f1_score(df_test['Category'], predictions))

f1 score: 0.0
